In [1]:
import os
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark import SparkFiles

import pyspark.sql.functions as F  ## similar to numpy for pandas
from pyspark.sql import types as T

%load_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
sc = SparkContext("local", "MyApplication")

<IPython.core.display.Javascript object>

In [3]:
sc.stop()

<IPython.core.display.Javascript object>

In [43]:
from sparkdl import readImages
from pyspark.sql.functions import lit

loc = os.path.abspath("")
data_loc = f"{loc}/images/"

img_dir = data_loc

# Read images and Create training & test DataFrames for transfer learning
jobs_df = readImages(img_dir + "/jobs").withColumn("label", lit(1))
zuckerberg_df = readImages(img_dir + "/zuckerberg").withColumn("label", lit(0))
jobs_train, jobs_test = jobs_df.randomSplit([0.6, 0.4])
zuckerberg_train, zuckerberg_test = zuckerberg_df.randomSplit([0.6, 0.4])

# dataframe for training a classification model
train_df = jobs_train.unionAll(zuckerberg_train)

# dataframe for testing the classification model
test_df = jobs_test.unionAll(zuckerberg_test)

<IPython.core.display.Javascript object>

In [44]:
train_df.show()

+--------------------+--------------------+-----+
|            filePath|               image|label|
+--------------------+--------------------+-----+
|file:/Users/glend...|[RGB, 222, 227, 3...|    1|
|file:/Users/glend...|[RGB, 276, 183, 3...|    1|
|file:/Users/glend...|[RGB, 185, 273, 3...|    1|
|file:/Users/glend...|[RGB, 268, 188, 3...|    1|
|file:/Users/glend...|[RGB, 223, 226, 3...|    1|
|file:/Users/glend...|[RGB, 183, 275, 3...|    1|
|file:/Users/glend...|[RGB, 177, 284, 3...|    1|
|file:/Users/glend...|[RGB, 218, 231, 3...|    1|
|file:/Users/glend...|[RGB, 183, 275, 3...|    1|
|file:/Users/glend...|[RGB, 220, 229, 3...|    1|
|file:/Users/glend...|[RGB, 245, 206, 3...|    1|
|file:/Users/glend...|[RGB, 193, 262, 3...|    1|
|file:/Users/glend...|[RGB, 189, 267, 3...|    1|
|file:/Users/glend...|[RGB, 276, 182, 3...|    0|
|file:/Users/glend...|[RGB, 179, 282, 3...|    0|
|file:/Users/glend...|[RGB, 183, 275, 3...|    0|
|file:/Users/glend...|[RGB, 183, 275, 3...|    0|


<IPython.core.display.Javascript object>

In [45]:
import tensorflow as tf

<IPython.core.display.Javascript object>

In [63]:
!conda install python=3.6

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done
Solving environment: done

## Package Plan ##

  environment location: /Users/glendont/anaconda3

  added / updated specs:
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-5.0.1    |   py36h6e48e2d_1           6 KB
    _ipyw_jlab_nb_ext_conf-0.1.0|           py36_0           4 KB
    alabaster-0.7.12           |           py36_0          18 KB
    anaconda-custom            |           py36_1           3 KB
    anaconda-client-1.7.2      |           py36_0         147 KB
    anaconda-navigator-1.9.12  |           py36_0         5.6 MB
    appnope-0.1.0              |   py36hf537a9a_0           8 KB
    appscript-1.1.0            |   py36h1de35cc_0         273 KB
    asn1crypto-1.3.0           |           py36_0         163 KB
    astroid-2.3.3

<IPython.core.display.Javascript object>

In [59]:
!pip install tensorflow==1.1



ERROR: Could not find a version that satisfies the requirement tensorflow==1.1 (from versions: 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3)
ERROR: No matching distribution found for tensorflow==1.1


<IPython.core.display.Javascript object>

In [46]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer


<IPython.core.display.Javascript object>

In [47]:
featurizer = DeepImageFeaturizer(
    inputCol="image", outputCol="features", modelName="InceptionV3"
)


<IPython.core.display.Javascript object>

In [48]:
lr = LogisticRegression(
    maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label"
)


<IPython.core.display.Javascript object>

In [58]:
strain = Pipeline(stages=[featurizer, lr])
strain.fit(train_df)

AttributeError: module 'tensorflow' has no attribute 'Session'

<IPython.core.display.Javascript object>

In [55]:
tf.compat.v1.Session()
tf.compat.v1.disable_eager_execution()
# p_model = p.fit(train_df)
# transformeddf = p_model.transform(df)

<IPython.core.display.Javascript object>

In [39]:
!pip install tf-nightly-2.0-preview

ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview


<IPython.core.display.Javascript object>

In [40]:
!ls -al

total 533440
drwxr-xr-x@ 15 glendont  staff        480 Apr 20 21:59 .
drwxr-xr-x@ 13 glendont  staff        416 Apr 20 21:59 ..
-rw-r--r--@  1 glendont  staff      10244 Apr 20 21:59 .DS_Store
drwxr-xr-x@  6 glendont  staff        192 Apr 20 21:56 .ipynb_checkpoints
-rw-rw-r--@  1 glendont  staff      66193 Jun  7  2018 Ecommerce_Customers.csv
-rw-r--r--   1 glendont  staff      39288 Apr 17 19:29 Natural Language Processing Tutorial.ipynb
-rw-r--r--   1 glendont  staff     171049 Apr 20 23:32 PySpark Fundamentals.ipynb
-rw-r--r--   1 glendont  staff      11959 Apr 20 21:53 PySpark Linear Regression.ipynb
-rw-r--r--   1 glendont  staff      53034 Apr 20 23:39 PySparkImage.ipynb
drwxr-xr-x@  4 glendont  staff        128 Apr 18 17:47 data
-rw-r--r--@  1 glendont  staff     110650 Apr 16 23:51 finaloutput.csv
-rw-r--r--   1 glendont  staff  228813984 Apr 20 22:12 flower_photos.tgz
drwxr-xr-x@  5 glendont  staff        160 Apr 20 21:59 images
-rw-r--r--@  1 glendont  staff   35770287 Apr 1

<IPython.core.display.Javascript object>

In [42]:
!tf_upgrade-v2 --infile PySparkImage.ipynb --outfile new.py

/bin/sh: tf_upgrade-v2: command not found


<IPython.core.display.Javascript object>

In [23]:
print(tensorflow.__version__)

2.0.0


<IPython.core.display.Javascript object>

In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.1.0
Uninstalling tensorflow-2.1.0:
  Would remove:
    /Users/glendont/anaconda3/bin/estimator_ckpt_converter
    /Users/glendont/anaconda3/bin/saved_model_cli
    /Users/glendont/anaconda3/bin/tensorboard
    /Users/glendont/anaconda3/bin/tf_upgrade_v2
    /Users/glendont/anaconda3/bin/tflite_convert
    /Users/glendont/anaconda3/bin/toco
    /Users/glendont/anaconda3/bin/toco_from_protos
    /Users/glendont/anaconda3/lib/python3.7/site-packages/tensorflow-2.1.0.dist-info/*
    /Users/glendont/anaconda3/lib/python3.7/site-packages/tensorflow/*
    /Users/glendont/anaconda3/lib/python3.7/site-packages/tensorflow_core/*
Proceed (y/n)? 